# Day 5 notebook

The objectives of this notebook are to practice

* checking if a graph is Eulerian
* finding a cycle in a graph
* using multiple inheritance in Python

In this notebook we will work with graph data structures.  Included with this notebook is a graph module (graph.py), which contains the abstract base class `DirectedGraph` and an adjacency list implementation, `AdjacencyListDirectedGraph`, from the Day 4 notebook.  We will use these graph interfaces and implementations throughout.

In [1]:
import graph

### PROBLEM 1: Check if a graph is Eulerian (3 POINTS)
First, write a function that takes as input a directed graph (i.e., an object that "is a" DirectedGraph), and returns whether the graph is Eulerian or not.  The function should assume that the graph is connected (i.e., there exists a path between every pair of vertices).

In [2]:
def is_eulerian(g):
    """Checks if a direct graph is Eulerian.  Assumes that g is connected.
    
    Args:
        g: a DirectedGraph object
    Returns:
        True if g is Eulerian, False otherwise.
    """
    ### BEGIN SOLUTION
    return all(g.outdegree(i) == g.indegree(i) for i in range(g.num_vertices()))
    ### END SOLUTION

In [3]:
# test for is_eulerian g1
g1 = graph.AdjacencyListDirectedGraph(4)
g1.add_edges([(0, 1), (1, 2), (1, 3), (2, 0), (2, 1), (3, 2)])
assert is_eulerian(g1) == True
print("SUCCESS: is_eulerian g1 test passed!")

SUCCESS: is_eulerian g1 test passed!


In [4]:
# test for is_eulerian g2 and g3
g2 = graph.AdjacencyListDirectedGraph(4)
g2.add_edges([(0, 1), (1, 2), (1, 3), (2, 0), (3, 2)])
assert is_eulerian(g2) == False
g3 = graph.AdjacencyListDirectedGraph(4)
g3.add_edges([(0, 1), (1, 2), (1, 3), (2, 1), (3, 2)])
assert is_eulerian(g3) == False
print("SUCCESS: is_eulerian g2 & g3 tests passed!")

SUCCESS: is_eulerian g2 & g3 tests passed!


In [5]:
# test for is_eulerian g4 and g5
g4 = graph.AdjacencyListDirectedGraph(2)
g4.add_edges([(0, 0), (0, 1), (1, 1), (1, 0)])
g5 = graph.AdjacencyListDirectedGraph(1)
g5.add_edges([(0, 0)])
assert is_eulerian(g4) == True
assert is_eulerian(g5) == True
print("SUCCESS: is_eulerian g4 & g5 tests passed!")

SUCCESS: is_eulerian g4 & g5 tests passed!


### PROBLEM 2: Finding a cycle in an Eulerian graph (3 POINTS)
Next, write a function that takes as input a Eulerian directed graph and a designated start vertex, and returns a cycle in the graph that starts and ends at the designated vertex and does use any edge more than once.  Note that this does **not** need to be an Eulerian cycle (which would traverse all edges).  Essentially, this function is tracing out the first cycle that would be found during the Eulerian cycle algorithm dicussed in the online lecture.

I encourage you to use the simple algorithm described in lecture, which is simply to traverse unused edges until you return to the start vertex.  You may find it useful to use a `set` object to keep track of edges that you have already used.

In [6]:
def find_cycle_in_eulerian_graph(g, start_vertex):
    """Finds an arbitrary cycle starting at a given vertex in a directed graph.
    Assumes that the graph is Eulerian, and thus that such a cycle exists."
    
    Args:
        g: a DirectedGraph object
        start_vertex: the index of the vertex from which the cycle should start
    Returns:
        A cycle represented by a list of indices of the vertices traversed, in order, by the cycle.  
        The first and last entries of the list will be identical and equal to start_vertex.
    """
    ### BEGIN SOLUTION
    cycle = [start_vertex]
    used_edges = set()
    while cycle[-1] != start_vertex or len(cycle) == 1:
        for edge in g.out_edges(cycle[-1]):
            if edge not in used_edges:
                used_edges.add(edge)
                cycle.append(edge[1])
                break
    return cycle
    ### END SOLUTION

In [7]:
# test for find_cycle_in_eulerian_graph g1
def is_cycle(g, path, vertex):
    """Returns True if path is a cycle in g that starts and ends with vertex."""
    # check for cyclical path with given start/end vertex
    if len(path) < 2 or path[0] != vertex or path[-1] != vertex:
        return False
    # check for distinct edges
    path_edges = set(zip(path[:-1], path[1:]))    
    if len(path_edges) != len(path) - 1:
        return False
    # check for existence of edges
    for i, j in path_edges:
        if not g.has_edge(i, j):
            return False
    return True

g1 = graph.AdjacencyListDirectedGraph(4)
g1.add_edges([(0, 1), (1, 2), (1, 3), (2, 0), (2, 1), (3, 2)])
assert is_cycle(g1, find_cycle_in_eulerian_graph(g1, 0), 0)
assert is_cycle(g1, find_cycle_in_eulerian_graph(g1, 3), 3)
print("SUCCESS: find_cycle_in_eulerian_graph g1 test passed!")

SUCCESS: find_cycle_in_eulerian_graph g1 test passed!


In [8]:
# test for find_cycle_in_eulerian_graph g4
g4 = graph.AdjacencyListDirectedGraph(2)
g4.add_edges([(0, 0), (0, 1), (1, 1), (1, 0)])
assert is_cycle(g4, find_cycle_in_eulerian_graph(g4, 0), 0)
assert is_cycle(g4, find_cycle_in_eulerian_graph(g4, 1), 1)
print("SUCCESS: find_cycle_in_eulerian_graph g4 test passed!")

SUCCESS: find_cycle_in_eulerian_graph g4 test passed!


In [9]:
# test for find_cycle_in_eulerian_graph g6
g6 = graph.AdjacencyListDirectedGraph(4)
g6.add_edges([(0, 1), (0, 2), (0, 3),
              (1, 0), (1, 2), (1, 3),
              (2, 0), (2, 1), (2, 3),
              (3, 0), (3, 1), (3, 2)])
assert is_cycle(g6, find_cycle_in_eulerian_graph(g6, 3), 3)
assert is_cycle(g6, find_cycle_in_eulerian_graph(g6, 0), 0)
print("SUCCESS: find_cycle_in_eulerian_graph g6 test passed!")

SUCCESS: find_cycle_in_eulerian_graph g6 test passed!


## Adding edge weights and vertex labels to graphs via "mixin" classes
In the Day 4 notebook, we saw how we could implement a graph data structure via Python classes and use inheritance to provide multiple alternative implementations.  Here, we will see how to easily add other features to graphs with "mixin" classes.  For example, we may want to allow edges to have weights and for vertices to have labels (e.g., read strings for overlap graphs).

Below is a "mixin" class `VertexLabeledDirectedGraph`, which allows you to "mix in" the feature of vertices having labels, with any particular implementation of a directed graph.  This eliminates the need to make vertex labeled versions of every possible graph implementation ahead of time.  Take a look at how `VertexLabeledDirectedGraph` is implemented.

In [10]:
class VertexLabeledDirectedGraph(graph.DirectedGraph):
    """A mixin class that allows for vertices in a directed graph to have labels."""
    
    def __init__(self, num_vertices):
        # call the next constructor in Python's Method Resolution Order
        super().__init__(num_vertices)
        self._vertex_labels = [None] * num_vertices

    def set_vertex_label(self, i, label):
        """Adds a label to vertex i."""
        self._vertex_labels[i] = label
       
    def vertex_label(self, i):
        """Returns the label of vertex i or None if it is not assigned a label"""
        return self._vertex_labels[i]

Below is how we would use this "mix in" class to add vertex labels to the adjacency list implementation of directed graphs.

In [11]:
# To "mix", simply inherit from multiple classes!
class VertexLabeledALDirectedGraph(graph.AdjacencyListDirectedGraph, VertexLabeledDirectedGraph):
    pass

And below is an example of how this new mixed class could be used.

In [12]:
g = VertexLabeledALDirectedGraph(3)
g.add_edge(0, 2)
g.add_edge(1, 2)
g.set_vertex_label(0, "the first vertex")
g.set_vertex_label(2, "the third vertex")
print(g)
print("The label of vertex 0 is", repr(g.vertex_label(0)))
print("The label of vertex 1 is", repr(g.vertex_label(1)))
print("The label of vertex 1 is", repr(g.vertex_label(2)))

DirectedGraph with 3 vertices and 2 edge(s):
[(0, 2), (1, 2)]
The label of vertex 0 is 'the first vertex'
The label of vertex 1 is None
The label of vertex 1 is 'the third vertex'


### PROBLEM 3: Adding functionality for edge weights via a "mixin" class (3 POINTS)
Now try your hand at defining another mixin class that allows for DirectedGraphs to have edge weights.  Consider using a `dict` to store the weights of edges and take note of the [`get`](https://docs.python.org/3/library/stdtypes.html#dict.get) method of `dict` objects. 

In [13]:
class EdgeWeightedDirectedGraph(graph.DirectedGraph):
    """A mixin class that allows for edges in a directed graph to have weights."""
    
    def __init__(self, num_vertices):
        ### BEGIN SOLUTION
        # call the next constructor in Python's Method Resolution Order
        super().__init__(num_vertices)
        self._edge_weights = dict()
        ### END SOLUTION

    def add_weighted_edge(self, i, j, weight):
        """Adds an edge to the graph with the given weight."""        
        ### BEGIN SOLUTION
        self.add_edge(i, j)
        self._edge_weights[(i, j)] = weight
        ### END SOLUTION        
       
    def edge_weight(self, i, j):
        """Returns the weight of edge (i, j) or None if it is not assigned a weight"""
        ### BEGIN SOLUTION
        return self._edge_weights.get((i, j), None)
        ### END SOLUTION

We can then use your class to add edge weight functionality to the adjacency list implementation:

In [14]:
class EdgeWeightedTSDirectedGraph(graph.AdjacencyListDirectedGraph, EdgeWeightedDirectedGraph):
    pass

Below is some example code that should run using the `EdgeWeightedTSDirectedGraph` class above.

In [15]:
g = EdgeWeightedTSDirectedGraph(10)
g.add_weighted_edge(2, 4, 11)
g.add_weighted_edge(6, 1, 9)
g.add_edge(1, 2) # add unweighted edge
print(g)
print("Weight of edge (2, 4):", g.edge_weight(2, 4))
print("Weight of edge (6, 1):", g.edge_weight(6, 1))
print("Weight of edge (1, 2):", g.edge_weight(1, 2))

DirectedGraph with 10 vertices and 3 edge(s):
[(1, 2), (2, 4), (6, 1)]
Weight of edge (2, 4): 11
Weight of edge (6, 1): 9
Weight of edge (1, 2): None


In [16]:
# tests for EdgeWeightedTSDirectedGraph constructor
assert EdgeWeightedTSDirectedGraph.__init__ != graph.DirectedGraph.__init__, "__init__ not overridden"
test_graph = EdgeWeightedTSDirectedGraph(10)
assert str(test_graph) == "DirectedGraph with 10 vertices and 0 edge(s):\n[]"
print("SUCCESS: EdgeWeightedTSDirectedGraph constructor passed all tests!")

SUCCESS: EdgeWeightedTSDirectedGraph constructor passed all tests!


In [17]:
# test for EdgeWeightedTSDirectedGraph add_weighted_edge
test_graph = EdgeWeightedTSDirectedGraph(10)
test_graph.add_weighted_edge(3, 5, 42)
assert test_graph.has_edge(3, 5)
test_graph.add_weighted_edge(2, 3, 43)
assert test_graph.has_edge(2, 3)
### BEGIN HIDDEN TESTS
test_graph.add_weighted_edge(0, 2, 44)
assert test_graph.has_edge(0, 2)
### END HIDDEN TESTS
print("SUCCESS: EdgeWeightedTSDirectedGraph add_weighted_edge passed all tests!")

SUCCESS: EdgeWeightedTSDirectedGraph add_weighted_edge passed all tests!


In [18]:
# test for EdgeWeightedTSDirectedGraph edge_weight
test_graph = EdgeWeightedTSDirectedGraph(10)
test_graph.add_weighted_edge(3, 5, 42)
assert test_graph.edge_weight(3, 5) == 42
test_graph.add_weighted_edge(2, 3, 43)
assert test_graph.edge_weight(2, 3) == 43
test_graph.add_edge(1, 2) # We should be able to add an edge without a weight
assert test_graph.edge_weight(1, 2) is None
### BEGIN HIDDEN TESTS
test_graph.add_weighted_edge(0, 2, 44)
assert test_graph.edge_weight(0, 2) == 44
### END HIDDEN TESTS
print("SUCCESS: EdgeWeightedTSDirectedGraph add_weighted_edge passed all tests!")

SUCCESS: EdgeWeightedTSDirectedGraph add_weighted_edge passed all tests!
